In [6]:
#%pip install protobuf==3.20.1
%pip install transformers[torch]
%pip install -q sentencepiece
%pip install datasets==2.13.1
%pip install evaluate
%pip install rouge_score
#%pip install git+https://github.com/huggingface/nlp.git@fix-bad-type-in-overflow-check

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 87.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
QPATH = "Quantlet/5-domain-pre-training"

In [9]:
import sys
IN_COLAB = 'google.colab' in sys.modules

import os

if IN_COLAB:
  os.chdir(f'/content/drive/MyDrive/ColabNotebooks/IRTG/Encode_the_Qode/Encode-the-Qode/{QPATH}')

sys.path.append('../4-qode2desc')

In [10]:
import pickle
import json
import re
import sys
from IPython.display import display

from tqdm import tqdm
import pandas as pd
import numpy as np
tqdm.pandas()


import matplotlib.pyplot as plt
import seaborn as sns

import torch
from torch.utils.data import  DataLoader
from transformers import AutoTokenizer, AutoModelWithLMHead, SummarizationPipeline
from transformers import AdamW
from datasets import load_dataset

from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq
)

import nltk
nltk.download('punkt')
import evaluate

import importlib
import datetime

from sklearn.model_selection import train_test_split

import analysis_modules
importlib.reload(analysis_modules)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


<module 'analysis_modules' from '/content/drive/MyDrive/ColabNotebooks/IRTG/Encode_the_Qode/Encode-the-Qode/Quantlet/5-domain-pre-training/../4-qode2desc/analysis_modules.py'>

In [13]:
model_name = "CodeT5"
SAMPLE = 'test'
os.environ["WANDB_DISABLED"] = "true"


# tokenization
encoder_max_length = 300
decoder_max_length = 15

RS = 42
LR = 5e-5

EPOCHS = 12
TRAIN_BATCH = 4
EVAL_BATCH  = 4

WARMUP_STEPS  = 500
WEIGHT_DECAY  = 0.1
LOGGING_STEPS = 100
SAVE_TOTAL_LIM = 1

LABEL_SMOOTHING  = 0.1
PREDICT_GENERATE = True

EVAL_COLUMNS = ['eval_loss',
                'eval_rouge1',
                'eval_rouge2',
                'eval_rougeL',
                'eval_rougeLsum',
                'eval_bleu',
                'eval_gen_len']

DATE = str(datetime.date.today())
analysis_name = model_name + '-'  + SAMPLE  + '-' + str(EPOCHS) + '-' + str(encoder_max_length) + '-' + str(TRAIN_BATCH)

analysis_name = analysis_name + '-' + DATE + '-' + 'v3'
print(analysis_name)

CodeT5-test-12-300-4-2023-09-26-v3


In [ ]:
print(analysis_name)
analysis_modules.scs_analyze(analysis_name=analysis_name,
                                    model_name=model_name,
                                    train_data_path=f'../../data/preprocessed/ArXiv/',
                                    train_data_name='arxiv_pretrain_train.json',
                                    val_data_path=f'../../data/preprocessed/ArXiv/',
                                    val_data_name='arxiv_pretrain_test.json',
                                    encoder_max_length=encoder_max_length,
                                    decoder_max_length=decoder_max_length,
                                    random_state=RS,
                                    eval_columns_list=EVAL_COLUMNS,
                                    learning_rate=LR,
                                    epochs=EPOCHS,
                                    train_batch=TRAIN_BATCH,
                                    eval_batch=EVAL_BATCH,
                                    warmup_steps=WARMUP_STEPS,
                                    weight_decay=WEIGHT_DECAY,
                                    logging_stes=LOGGING_STEPS,
                                    save_total_lim=SAVE_TOTAL_LIM,
                                    label_smooting=LABEL_SMOOTHING,
                                    predict_generate=PREDICT_GENERATE,
                                    save_strategy='epoch',
                                    load_best_model_at_end=False)

CodeT5-test-12-300-4-2023-09-26-v3
CodeT5-test-12-300-4-2023-09-26-v3
cuda


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1479: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


cuda


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


   eval_loss  eval_rouge1  eval_rouge2  eval_rougeL  eval_rougeLsum  \
0      5.803         0.15        0.044        0.133           0.133   

   eval_bleu  eval_gen_len  
0      0.003        18.456  


Step,Training Loss
100,5.284400
200,4.523100
300,4.122400
400,3.970800
500,3.816600
600,3.760900
700,3.708800
800,3.751900
900,3.687900
1000,3.650000


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-14-0f1867e28398>", line 2, in <cell line: 2>
    analysis_modules.scs_analyze(analysis_name=analysis_name,
  File "/content/drive/MyDrive/ColabNotebooks/IRTG/Encode_the_Qode/Encode-the-Qode/Quantlet/5-domain-pre-training/../4-qode2desc/analysis_modules.py", line 264, in scs_analyze
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 1553, in train
    return inner_training_loop(
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 1942, in _inner_training_loop
    self._maybe_log_save_evaluate(tr_loss, model, trial, epoch, ignore_keys_for_eval)
  File "/usr/local/lib/python3.10/dist-packages/transformers/trainer.py", line 2265, in _maybe_log_save_evaluate
    self._save_checkpoint(model, trial, metrics=metrics)
  File "